In [24]:
import easyocr
import numpy as np
import cv2
import pandas as pd
import string
from collections import Counter
from nltk.corpus import stopwords
import pandas as pd
import cv2
#import pytesseract
from pytesseract import Output
import re
import pickle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package stopwords to /Users/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
prepositions = [
                                "aboard", "about", "above", "across", "after", "against", "along", "amid", 
                                "among", "around", "as", "at", "before", "behind", "below", "beneath", 
                                "beside", "between", "beyond", "but", "by", "concerning", "considering", 
                                "despite", "down", "during", "except", "for", "from", "in", "inside", 
                                "into", "like", "near", "of", "off", "on", "onto", "out", "outside", 
                                "over", "past", "regarding", "round", "since", "through", "to", "toward", 
                                "under", "underneath", "until", "unto", "up", "upon", "with", "within", "without"
                              ]
exclude_words = ['de', 'en', 'al', 'alla', 'con','e','di','ai']
filter_words = stopwords.words('english') + prepositions #+ exclude_words

## Common Menu Words

In [7]:
df_dish = pd.read_csv('Dish.csv')

df_dish_mod = df_dish[df_dish.menus_appeared > 350].drop(columns = 'description')\
.sort_values(by = ['menus_appeared','times_appeared'], ascending = False).reset_index(drop=True)[:250]
dish_items = []
# for i in range(len(df_dish_mod)):
#     dish_items.append(df_dish_mod['name'][i])        
dish_items += american_menu_items
dish_items += italian_menu_items
dish_items += mexican_menu_items
dish_items += asian_menu_items
    
individual_words = [word.lower() for item in dish_items for word in item.split()]
word_counts = Counter(individual_words)
common_menu_words = [word for word, freq in word_counts.most_common(160)]#if word not in filter_words]
[common_menu_words.append(i) for i in ['feast','grill','grilled','skillet']]

[None, None, None]

In [3]:
italian_menu_items = [
    'Panna Cotta', 'Cotoletta alla Milanese', 'Ravioli', 'Cannoli', 'Lasagna', 'Italian Cream Cake',
    'Osso Buco', 'Bruschetta', 'Focaccia', 'Antipasto', 'Ossobuco alla Milanese', 'Eggplant Parmigiana',
    'Italian Sausage', 'Prosciutto e Melone', 'Pasta e Fagioli', 'Cioppino', 'Tuscan Chicken',
    'Vitello Tonnato', 'Zuppa Toscana', 'Tiramisu', 'Risotto ai Frutti di Mare', 'Muffuletta',
    'Mozzarella in Carrozza', 'Spaghetti Carbonara', 'Biscotti', 'Ricotta Cheesecake', 'Pasta Fagioli',
    'Zeppole', 'Penne alla Vodka', 'Saltimbocca alla Romana', 'Spaghetti alla Puttanesca', 'Pollo alla Piccata',
    'Carpaccio', 'Caponata', 'Prosciutto', 'Spaghetti Bolognese', 'Lasagna alla Bolognese', 'Marsala',
    'Torta Caprese', 'Ziti', 'Burrata', 'Fettuccine Alfredo', 'Spaghetti and Meatballs', 'Vitello tonnato',
    'Pasta Pomodoro', 'Zabaglione', 'Porchetta', 'Baccala', 'Fusilli', 'Fritto Misto', 'Panzanella',
    'Orecchiette con le Cime di Rapa', 'Spaghetti alla Carbonara', 'Frutti di Mare', 'Pesto Genovese',
    'Stracciatella Soup', 'Chicken Parmigiana', 'Gelato', 'Pasta Primavera', 'Calzone', 'Sfogliatella',
    'Tiramisu Cake', 'Saltimbocca', 'Chicken Piccata', 'Gnocchi', 'Polenta', 'Limoncello',
    'Pollo alla Marsala', 'Minestrone', 'Baked Ziti', 'Grappa', 'Negroni', 'Cassata',
    'Minestrone Soup', 'Risotto alla Milanese', 'Risotto ai Funghi', 'Anisette', 'Espresso', 'Escarole Soup',
    'Bagna Cauda', 'Tortellini', 'Bistecca alla Fiorentina', 'Ribollita', 'Chicken Marsala',
    'Pesto alla Genovese', 'Pasta Aglio e Olio', 'Manicotti', 'Italian Wedding Soup', 'Arancini',
    'Amaretto', 'Caprese Salad', 'Pizza Margherita', 'Rigatoni', 'Affogato', 'Pappa al Pomodoro',
    'Frittata', 'Pizza Napoletana'
]


In [4]:
mexican_menu_items = [
    "Tacos al Pastor", "Chiles Rellenos", "Ceviche", "Tamales", "Pozole", "Guacamole", 
    "Enchiladas", "Chimichangas", "Churros", "Flan", "Empanadas", "Sopes", "Menudo", 
    "Mole Sauce", "Pico de Gallo", "Carnitas", "Carne Asada", "Fajitas", "Quesadillas", 
    "Nachos", "Tostadas", "Burritos", "Chorizo", "Tortas", "Pupusas", "Chilaquiles", 
    "Chiles en Nogada", "Horchata", "Carnitas", "Huevos Rancheros", "Carne en su Jugo", 
    "Enmolada", "Tlayuda", "Atole", "Tamarind Agua Fresca", "Bionicos", "Authentic Tacos Al Pastor",
    "Tamales De Puerco", "Mexican Corn On The Cob", "Mexican Sopes", "Churros", 
    "Authentic Mexican Torta", "Papas Rellenas", "Chef John's Horchata", "Birria De Chivo Estilo Jalisco",
    "Conchas", "Pozole In A Slow Cooker", "Avocado Paletas", "Creamy Caramel Flan", 
    "Classic Tres Leches Cake", "Fresas con Crema", "Conchas", "Mexican Cinnamon Cookies", 
    "Homemade Churros", "Strawberry Basil and Honey Paletas","Sopaipillas", "Mayan Chocolate Biscotti", 
    "Margarita Tres Leches Cake","Dulce de Leche Banana Cream Pie","Almond Coconut Flans", 
    "Pumpkin Cheesecake Empanadas","Cherry Biscochitos","Shortcut Tres Leches Cake","Arroz con Leche",
    "Sopaipilla Stars", "Pastelitos de Boda","Chilaquiles", "Mexican Egg Bake", 
    "Black Bean Huevos Rancheros", "Chorizo Breakfast Bowls","Migas", "Mexican Hash", "Carne Asada Breakfast Burrito", 
    "Papas Con Chorizo", "Microwave Longaniza Con Huevos", "Crispy Rolled Breakfast Burrito", "Migas Breakfast Tacos",
    "Michelada"
]

In [5]:
asian_menu_items = [
    'Halo-Halo', 'Bakso', 'Palak Paneer', 'Biryani', 'Tandoori Chicken','Nasi Goreng', 'Bao Buns', 'Tteokbokki', 
    'Rendang', 'Gỏi cuốn','Tempura', 'Hainanese Chicken', 'Satay', 'Hummus', 'Chicken Adobo',
    'Tonkatsu', 'Pani Puri', 'Laksa', 'Murtabak', 'Banh Mi','Baozi', 'Mango Sticky Rice', 'Soba', 'Beef Rendang', 
    'Onigiri', 'Naan', 'Char Siu', 'Egg Fried Rice', 'Goi Cuon', 'Udon','Hot Pot', 'Lechon', 'Chilli Crab', 
    'Chicken Rice', 'Mapo Tofu','Samosa', 'Som Tam', 'Massaman Curry', 'Roti Canai', 'Daal Makhani',
    'Butter Chicken', 'Soto Ayam', 'Tom Yum Soup', 'Sinigang', 'Galbi','Bak Kut Teh', 'Sweet and Sour Pork', 
    'Katsu Curry', 'Char Kway Teow', 'Bibimbap','Kung Pao Chicken', 'Peking Duck', 'Soondubu Jjigae', 'Curry Laksa', 
    'Kimchi','Kare-Kare', 'Cao Lau', 'Jiaozi', 'Yakitori', 'Xiao Long Bao','Spring Rolls', 'Tom Yum Goong', 
    'Century Egg', 'Pancit', 'Gimbap', 'Okonomiyaki', 'Pav Bhaji', 'Miso Soup', 'Shawarma', 'Nasi Lemak',
    'Rogan Josh', 'Vada Pav', 'Chicken Tikka Masala', 'Hainanese Chicken Rice', 'Takoyaki',
    'Dim Sum', 'Pho', 'Fish Head Curry', 'Green Curry', 'Balut','Japchae', 'Adobo', 'Falafel', 'Wonton Soup', 
    'Chole Bhature', 'Ramen', 'Pad Thai', 'Congee', 'Bun Cha', 'Bánh Xèo','Bulgogi', 'Sushi'
]


In [6]:
american_menu_items = [
    "Tater tots", "Burgers", "S'mores", "Biscuits", 
    "Pulled pork", "Shrimp and grits", "Key lime pie", "Lobster rolls", "Fried Green Tomatoes", 
    "lox bagel", "Cheeseburger", "Chocolate chip cookie", "Banana bread", "Cornbread", 
    "Cuban bread", "Frybread", "Texas toast", "American cheese","Avocado toast", "Indian Fry Bread", 
    "Turkey Shepherd's Pie","Steak and shrimp kabobs","Mashed Potatoes", "French Fries", "Cracker Jack", 
    "Congealed salads and casseroles", "Frozen orange juice", "Beef Wellington", "Smoothies", 
    "Lunchables", "Cupcakes", "Spare ribs", "Chicken wings", "Skewered beef", 
    "Fried wontons", "Fried shrimp", "Chicken fingers", "Crab Rangoon", "Gouda and Swiss cheeses", "Apple pie", 
    "Ground beef", "Sausage", "Cayenne pepper", "Hot Italian sausage", "Eggs","Rice",'Hamburger', 'Hot Dog', 
    'Chicken Nuggets', 'Pizza', 'Steak', 'Fried Chicken', 'BBQ Ribs', 'Lobster Roll', 'Clam Chowder', 'Buffalo Wings',
    'Philly Cheesesteak', 'Reuben Sandwich', 'Cobb Salad', 'Caesar Salad', 'Chicken Caesar Wrap', 'BLT Sandwich', 
    'Grilled Cheese Sandwich', 'Tuna Salad Sandwich', 'Egg Salad Sandwich', 'Chicken Alfredo', 'Spaghetti Bolognese',
    'Lasagna', 'Mac and Cheese', 'Fish and Chips', 'Shrimp Scampi', 'Crab Cakes', 'Lobster Bisque', 
    'Chicken Noodle Soup', 'Chicken and Waffles', 'Pancakes', 'Waffles', 'French Toast', 'Eggs Benedict', 
    'Bacon and Eggs', 'Sausage and Eggs', 'Omelette', 'Bagel and Cream Cheese', 
    'Donut', 'Cherry Pie', 'Pumpkin Pie', 'Cheesecake', 'Brownies', 'Ice Cream Sundae', 'Banana Split', 'Milkshake', 
    'Root Beer Float', 'Lemonade', 'Iced Tea', 'Hot Chocolate', 'Coffee', 'Espresso', 
    'Cappuccino', 'Latte', 'Mocha', 'Green Tea', 'Black Tea', 'Herbal Tea', 'Water', 'Orange Juice', 'Apple Juice', 
    'Grape Juice', 'Chocolate Milk', 'Strawberry Milkshake', 'Vanilla Milkshake', 'Chocolate Milkshake', 
    'Mimosa'
]

In [12]:
with open('dish_items.pkl', 'wb') as f:
    pickle.dump(dish_items, f)

In [11]:
with open('common_menu_words.pkl', 'wb') as f:
    pickle.dump(common_menu_words, f)

In [2]:
# Load the list from the file
with open('dish_items.pkl', 'rb') as f:
    dish_items = pickle.load(f)

# Load the list from the file
with open('common_menu_words.pkl', 'rb') as f:
    common_menu_words = pickle.load(f)

In [21]:
start_menu = 1

stop_words = set(stopwords.words('english'))

prepositions = ['aboard', 'across', 'along', 'amid', 'among', 'around', 'behind', 'beneath', 'beside', 'beyond', \
                'concerning', 'considering', 'despite', 'except', 'inside', 'like', 'near', 'onto', 'outside', 'past',\
                'regarding', 'round', 'since', 'throughout', 'toward', 'underneath', 'unto', 'upon', 'within', 'without']

food_categories = ['gyros','sandwiches','toppings','sauces','orders','shareables','drinks', 'beverages', 'appetizers',\
                   'starters', 'salads', 'soups', 'entrees','entrées', 'main courses', 'desserts', 'sweets', 'sandwiches',\
                   'pizzas', 'burgers', 'pastas', 'seafood', 'steaks', 'vegetarian/vegan', 'sides', 'side dishes',\
                   'breakfast/brunch', "kids' menu", 'combos', 'platters', 'specials of the day', 'snacks', 'bar snacks',\
                   'sushi', 'japanese cuisine', 'dim sum', 'chinese cuisine', 'tacos', 'mexican cuisine', 'tapas',\
                   'spanish cuisine', 'mezze', 'mediterranean cuisine', 'bento', 'japanese lunch boxes', 'cuisines',\
                   'wraps/rolls', 'all you can eat', 'family style', 'shared dishes','sauce']

non_words = ['available',"choice","choices","add","oz","available","w",'served','side','additional','contains','allergy','.com',\
            'pizzeria','salads','soups','side items','appetizers','create','classics','your own','plates','extra']

non_words += food_categories

def count_uppercase_letters(text):
    return sum(1 for i in text if i.isupper())
def count_numerical_chars(text):
    return sum(1 for i in text if i.isdigit())
def count_punctuation(text):
    return sum(1 for i in text if i in string.punctuation and i not in [',', '.', '$',':'])
def count_common_menu_words(text):
    words = re.findall(r'\b[a-zA-Z]+\b', text)  # Extract words containing only alphabetical characters
    lowercase_words = [word.lower() for word in words]  # Convert all words to lowercase
    return sum(word in common_menu_words for word in lowercase_words)
def count_consecutive_periods(text):
    return len(re.findall(r'\.{2,}', text))
def non_words_count(text):
    words = re.findall(r'\b[a-zA-Z]+\b', text)  # Extract words containing only alphabetical characters
    lowercase_words = [word.lower() for word in words]  # Convert all words to lowercase
    return sum(lowercase_words.count(non_word) for non_word in non_words)
def stop_words_count(text):
    words = re.findall(r'\b[a-zA-Z]+\b', text)  # Extract words containing only alphabetical characters
    lowercase_words = [word.lower() for word in words]  # Convert all words to lowercase
    return sum(lowercase_words.count(stop_word) for stop_word in stop_words)
def prepositions_count(text):
    words = re.findall(r'\b[a-zA-Z]+\b', text)  # Extract words containing only alphabetical characters
    lowercase_words = [word.lower() for word in words]  # Convert all words to lowercase
    return sum(lowercase_words.count(preposition) for preposition in prepositions)

# Create a reader for English language
reader = easyocr.Reader(['en'])

files1 = ['menu_2p2.jp2','olive-garden-menu-6.jpeg','Red-Lobster-menu-1.jpeg','Red-Lobster-menu-2.jpeg',\
          'Red-Lobster-menu-5.jpeg','mexican-menu-1.jpeg','mexican-menu-2.jpeg','mexican-menu-3.jpeg',\
        'mexican-menu-4.jpeg','american-menu-1.jpeg','american-menu-2.jpeg','ivy-menu-1.jpeg','ivy-menu-2.jpeg']

files2 = ['topgolf_menu.jpeg','eggcited_menu.jpeg','steak-n-shake.jpeg','tgi_menu.jpeg','portillos_menu.jpeg',\
         'portillos_menu2.jpeg','meatheads_menu.jpeg','meatheads_menu2.jpeg','PitaPit_menu.jpeg','PitaPit_menu2.jpeg']

files = files1 + files2

for c,file in enumerate(files):    
    # Read the images using OpenCV
    image = cv2.imread(file)
    
    # Convert the image to RGB (OpenCV loads images in BGR format)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Get the dimensions of the image
    height_img, width_img, _ = image.shape
    area_img = width_img*height_img
    
    # Perform OCR on the image
    result = reader.readtext(image)

    # Prepare a list to store the data
    data = []

    # Loop through the results
    for (bbox, text, prob) in result:
        # unpack the bounding box
        (tl, tr, br, bl) = bbox
        tl = (int(tl[0]), int(tl[1]))
        tr = (int(tr[0]), int(tr[1]))
        br = (int(br[0]), int(br[1]))
        bl = (int(bl[0]), int(bl[1]))        
        
        # calculate the center coordinates
        center_x = int((tl[0] + tr[0] + br[0] + bl[0]) / 4)/(width_img)
        center_y = int((tl[1] + tr[1] + br[1] + bl[1]) / 4)/(height_img)

        # Compute the width and height of the bounding box
        width = np.sqrt((br[0]-bl[0])**2 + (br[1]-bl[1])**2)/(width_img)
        height = np.sqrt((tr[0]-br[0])**2 + (tr[1]-br[1])**2)/(height_img)

        # Compute the area of the bounding box
        area = (width * height)

        # Append the text, probability, bbox coordinates, width, height, and area to the data list
        data.append([text, center_x, center_y, width, height, area])

    # data_2 = [(entry[0], entry[6], entry[7], entry[8]) for entry in data]
    
    # Create a pandas DataFrame from the data
    df = pd.DataFrame(data, columns=["Text", "center_x" ,"center_y", "width", "height", "area"])

    # Select only 'Text' and 'Height' columns
    df.insert(0, 'Menu', c + start_menu)
    df = df[['Menu','Text', "center_x" ,"center_y", "width", 'height','area']]
#     df = df.astype({'width':'int'})
#     df = df.astype({'height':'int'})
#     df = df.astype({'area':'int'})
    df['uppercase'] = df['Text'].apply(count_uppercase_letters)

    df['Text'] = df['Text'].str.strip()
    # Add character count
    df['chars'] = df['Text'].apply(len)

    # Add word count
    df['words'] = df['Text'].apply(lambda x: len(x.split()))
    
    df['contains_dish'] = df['Text'].apply(lambda x: 1 if any(item in x for item in dish_items) else 0)

    df['common_words'] = df['Text'].apply(count_common_menu_words)    
    df['non_words'] = df['Text'].apply(lambda x: non_menu_words(x, non_words))

    df['periods'] = df['Text'].apply(lambda x: x.count('.'))
    df['period_btw_numbers'] = df['Text'].apply(lambda x: bool(re.search(r'\d\.\d', x))).astype(int)
    df['number_end'] = df['Text'].apply(lambda x: bool(re.search(r'\d$', x))).astype(int)
    df['numbers'] = df['Text'].apply(count_numerical_chars)
    df['commas'] = df['Text'].apply(lambda x: x.count(','))
    df['exclamation'] = df['Text'].apply(lambda x: x.count('!'))
    df['question'] = df['Text'].apply(lambda x: x.count('?'))
    df['colons'] = df['Text'].apply(lambda x: x.count(':'))
    df['underscores'] = df['Text'].apply(lambda x: x.count('_'))
    df['dollar'] = df['Text'].apply(lambda x: x.count('$'))
    df['punctuation'] = df['Text'].apply(count_punctuation)
    df['2_periods_cnt'] = df['Text'].apply(count_consecutive_periods)
    df['stop_words'] = df['Text'].apply(stop_words_count)
    df['prepositions'] = df['Text'].apply(prepositions_count)

    df['Item'] = np.zeros(len(df))
    df['Item'] = df['Item'].astype('int')
    if c > 0:
        dfs = pd.concat([dfs,df], ignore_index=True)
    else:
        dfs = df.copy()
dfs

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


,Menu,Text,center_x,center_y,width,height,area,uppercase,chars,words,contains_dish,common_words,non_words,periods,period_btw_numbers,number_end,numbers,commas,exclamation,question,colons,underscores,dollar,punctuation,2_periods_cnt,stop_words,prepositions,Item
0,1,Soups,0.165238,0.062353,0.254286,0.050000,0.012714,1,5,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,16,0.091905,0.102549,0.063810,0.021373,0.001364,0,2,1,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0
2,1,Tom Yum Chicken*.,0.391429,0.104118,0.470476,0.024706,0.011624,3,17,3,0,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,1,2.95,0.877143,0.106471,0.104286,0.019412,0.002024,0,4,1,0,0,0,1,1,1,3,0,0,0,0,0,0,0,0,0,0,0
4,1,spicy & sour w/ mushrooms,0.473810,0.128431,0.540476,0.023529,0.012717,0,25,5,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0
5,1,L5.95,0.858095,0.129216,0.138571,0.019608,0.002717,1,5,1,0,0,0,1,1,1,3,0,0,0,0,0,0,0,0,0,0,0
6,1,in lemon grass broth,0.407143,0.152549,0.406667,0.025098,0.010207,0,20,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,1,17,0.088095,0.170196,0.059048,0.020392,0.001204,0,2,1,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0
8,1,"Tom Yum Goong (shrimp) ,3.95",0.543810,0.173725,0.781905,0.030588,0.023917,3,28,5,1,3,0,1,1,1,3,1,0,0,0,0,0,2,0,0,0,0
9,1,L6.95,0.855238,0.196275,0.140476,0.019412,0.002727,1,5,1,0,0,0,1,1,1,3,0,0,0,0,0,0,0,0,0,0,0


## BERT

In [25]:
#df_basic = df_basic_old
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def texts_to_vectors(texts):
    print("Starting texts_to_vectors function...")
    
    vectors = []
    for cnt, text in enumerate(texts):
        try:            
            inputs = tokenizer([text], return_tensors="pt", truncation=True, padding=True)
            with torch.no_grad():
                outputs = model(**inputs)
            embeddings = outputs.last_hidden_state
            mask = inputs.attention_mask
            masked_embeddings = embeddings * mask.unsqueeze(-1)
            summed = torch.sum(masked_embeddings, 1)
            summed_mask = torch.clamp(mask.sum(1), min=1e-9)
            mean_pooled = summed / summed_mask.unsqueeze(-1)
            vectors.append(mean_pooled[0].numpy())
        except Exception as e:
            print(f"Error encountered while processing text {cnt}: {e}")
            continue
        # Print progress update every 400 samples
        if cnt % 400 == 0:
            percentage = (cnt / len(texts)) * 100
            print(f"Processing text {cnt} of {len(texts)} ({percentage:.2f}% complete)")
    
    print("Finished converting texts to vectors.")
    return np.array(vectors)

vectors = texts_to_vectors(dfs['Text'].tolist())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Starting texts_to_vectors function...
Processing text 0 of 2576 (0.00% complete)
Processing text 400 of 2576 (15.53% complete)
Processing text 800 of 2576 (31.06% complete)
Processing text 1200 of 2576 (46.58% complete)
Processing text 1600 of 2576 (62.11% complete)
Processing text 2000 of 2576 (77.64% complete)
Processing text 2400 of 2576 (93.17% complete)
Finished converting texts to vectors.


In [31]:
df_vectors = pd.DataFrame(vectors, columns=[f'vector_{i}' for i in range(vectors.shape[1])])

# dfs_dropped_text = dfs.drop(columns=['Text'])
dfs_bert = pd.concat([dfs, df_vectors], axis=1)

dfs_bert.to_csv('Combined_menu_23_items_bert.csv',index = False)

In [32]:
dfs_bert.head()

,Menu,Text,center_x,center_y,width,height,area,uppercase,chars,words,contains_dish,common_words,non_words,periods,period_btw_numbers,number_end,numbers,commas,exclamation,question,colons,underscores,dollar,punctuation,2_periods_cnt,stop_words,prepositions,Item,vector_0,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,vector_8,vector_9,vector_10,vector_11,vector_12,vector_13,vector_14,vector_15,vector_16,vector_17,vector_18,vector_19,vector_20,vector_21,vector_22,vector_23,vector_24,vector_25,vector_26,vector_27,vector_28,vector_29,vector_30,vector_31,vector_32,vector_33,vector_34,vector_35,vector_36,vector_37,vector_38,vector_39,vector_40,vector_41,vector_42,vector_43,vector_44,vector_45,vector_46,vector_47,vector_48,vector_49,vector_50,vector_51,vector_52,vector_53,vector_54,vector_55,vector_56,vector_57,vector_58,vector_59,vector_60,vector_61,vector_62,vector_63,vector_64,vector_65,vector_66,vector_67,vector_68,vector_69,vector_70,vector_71,vector_72,vector_73,vector_74,vector_75,vector_76,vector_77,vector_78,vector_79,vector_80,vector_81,vector_82,vector_83,vector_84,vector_85,vector_86,vector_87,vector_88,vector_89,vector_90,vector_91,vector_92,vector_93,vector_94,vector_95,vector_96,vector_97,vector_98,vector_99,vector_100,vector_101,vector_102,vector_103,vector_104,vector_105,vector_106,vector_107,vector_108,vector_109,vector_110,vector_111,vector_112,vector_113,vector_114,vector_115,vector_116,vector_117,vector_118,vector_119,vector_120,vector_121,vector_122,vector_123,vector_124,vector_125,vector_126,vector_127,vector_128,vector_129,vector_130,vector_131,vector_132,vector_133,vector_134,vector_135,vector_136,vector_137,vector_138,vector_139,vector_140,vector_141,vector_142,vector_143,vector_144,vector_145,vector_146,vector_147,vector_148,vector_149,vector_150,vector_151,vector_152,vector_153,vector_154,vector_155,vector_156,vector_157,vector_158,vector_159,vector_160,vector_161,vector_162,vector_163,vector_164,vector_165,vector_166,vector_167,vector_168,vector_169,vector_170,vector_171,vector_172,vector_173,vector_174,vector_175,vector_176,vector_177,vector_178,vector_179,vector_180,vector_181,vector_182,vector_183,vector_184,vector_185,vector_186,vector_187,vector_188,vector_189,vector_190,vector_191,vector_192,vector_193,vector_194,vector_195,vector_196,vector_197,vector_198,vector_199,vector_200,vector_201,vector_202,vector_203,vector_204,vector_205,vector_206,vector_207,vector_208,vector_209,vector_210,vector_211,vector_212,vector_213,vector_214,vector_215,vector_216,vector_217,vector_218,vector_219,vector_220,vector_221,vector_222,vector_223,vector_224,vector_225,vector_226,vector_227,vector_228,vector_229,vector_230,vector_231,vector_232,vector_233,vector_234,vector_235,vector_236,vector_237,vector_238,vector_239,vector_240,vector_241,vector_242,vector_243,vector_244,vector_245,vector_246,vector_247,vector_248,vector_249,vector_250,vector_251,vector_252,vector_253,vector_254,vector_255,vector_256,vector_257,vector_258,vector_259,vector_260,vector_261,vector_262,vector_263,vector_264,vector_265,vector_266,vector_267,vector_268,vector_269,vector_270,vector_271,vector_272,vector_273,vector_274,vector_275,vector_276,vector_277,vector_278,vector_279,vector_280,vector_281,vector_282,vector_283,vector_284,vector_285,vector_286,vector_287,vector_288,vector_289,vector_290,vector_291,vector_292,vector_293,vector_294,vector_295,vector_296,vector_297,vector_298,vector_299,vector_300,vector_301,vector_302,vector_303,vector_304,vector_305,vector_306,vector_307,vector_308,vector_309,vector_310,vector_311,vector_312,vector_313,vector_314,vector_315,vector_316,vector_317,vector_318,vector_319,vector_320,vector_321,vector_322,vector_323,vector_324,vector_325,vector_326,vector_327,vector_328,vector_329,vector_330,vector_331,vector_332,vector_333,vector_334,vector_335,vector_336,vector_337,vector_338,vector_339,vector_340,vector_341,vector_342,vector_343,vector_344,vector_345,vector_346,vector_347,vector_348,vector_349

In [22]:
df_labeled = pd.read_csv('Combined_menu_23_items.csv')
dfs.Item = df_labeled.Item
dfs.to_csv('Combined_menu_23_items.csv',index = False)

In [117]:
import pandas as pd

# Assuming df is your DataFrame and 'column_name' is the name of the column containing the text
df = pd.DataFrame({
    'column_name': ['Petite Maine lobster tails, split and 26', 'Another dish', 'Yet another dish']
})

menu_items = ['lobster tail', 'another menu item', 'yet another menu item']

# Create a new column 'contains_item' that is True if the 'column_name' contains any of the menu items
df['contains_item'] = df['column_name'].apply(lambda x: any(item in x for item in menu_items))

print(df)


                                column_name  contains_item
0  Petite Maine lobster tails, split and 26           True
1                              Another dish          False
2                          Yet another dish          False


In [77]:
# Use pytesseract to get the information about each detected region
data = pytesseract.image_to_data(gray_image, output_type=Output.DICT)

# Prepare an empty dictionary to hold lines of text
lines = {}

# Iterate over each detected region
for i in range(len(data["text"])):
    # Get the information about this region
    x, y, width, height, text = data["left"][i], data["top"][i], data["width"][i], data["height"][i], data["text"][i]

    # Filter out regions that don't contain any text
    if len(text.strip()) == 0:
        continue

    # Group words into lines based on their y-coordinate
    line_key = round(y / height)

    if line_key in lines:
        # Append this word to an existing line
        lines[line_key][0].append(text)
        # Update the font size if this word is larger
        if height > lines[line_key][1]:
            lines[line_key][1] = height
    else:
        # Start a new line with this word
        lines[line_key] = [[text], height]

# Print each line and its font size
for line in lines.values():
    print(f'Font size: {line[1]}, Text: {" ".join(line[0])}')

Font size: 23, Text: BREAKFAST
Font size: 22, Text: Served every and 9am GARCIA’S
Font size: 16, Text: Saturday Sunday from 3pm
Font size: 16, Text: to Enchiladas Suizas
Font size: 14, Text: $12
Font size: 14, Text: ‘Two chicken enchiladas with green tomatillo $12
Font size: 11, Text: large topped
Font size: 22, Text: salsa and served with Mexican beans Vegetarian
Font size: 7, Text: sour
Font size: 21, Text: cream, rice, $12 Red Menudo Chilaquiles
Font size: 18, Text: and SOUTH BORDER bowl
Font size: 16, Text: eggs. Mexican Omelette
Font size: 31, Text: Huevos Rancheros your
Font size: 28, Text: A tortilla tower, refried ranchera, Omelette flour tortillas. protein Breakfast
Font size: 9, Text: crispy easy eggs
Font size: 28, Text: beans, topped CANTINA & GRILL our version
Font size: 16, Text: rice over Burrito
Font size: 16, Text: with salsa melted cheese crowned with $11 Divorciados $11)
Font size: 15, Text: sour cream queso $12 combination of red served with spicy peppers, $10
Font 

- Menu 1: Asian menu, "menu_2p2"
- Menu 2: Olive Garden, "olive-garden-menu-6"
- Menu 3: Red Lobster, "Red-Lobster-menu-1"
- Menu 4: Red Lobster, "Red-Lobster-menu-2"
- Menu 5: Red Lobster, "Red-Lobster-menu-5"
- Menu 6: Mexican menu,'mexican-menu-1'
- Menu 7: Mexican menu, 'mexican-menu-2'
- Menu 8: Mexican menu,'mexican-menu-3',
- Menu 9: Mexican menu,'mexican-menu-4',
- Menu 10: American menu,'american-menu-1',
- Menu 11: American menu,'american-menu-2'
- Menu 12: Random menu,'ivy-menu-1'
- Menu 13: Random menu,'ivy-menu-2'
- Menu 14: Top Golf, 'topgolf_menu'
- Menu 15: Eggcited, 'eggcited_menu'
- Menu 15: Steak & Shake, 'steak-n-shake'
- Menu 17: TGI Friday's, 'tgi_menu'
- Menu 18: Portillos, 'portillos_menu'
- Menu 19: Portillos, 'portillos_menu2'
- Menu 20: Meatheads, 'meatheads_menu'
- Menu 21: Meatheads, 'meatheads_menu2'
- Menu 22: PitaPit, 'PitaPit_menu'
- Menu 23: PitaPit, 'PitaPit_menu2'

Make another feature indicating the type of cuisine? This might be something the user would input to help the model make more accurate classification. 

In [58]:
df_cars = pd.read_csv('/Users/alex/Data_Science/Used_Cars/Car_Price_Prediction/car_data_sorted_Full.csv')

In [63]:
df_cars[(2010 > df_cars.Year) & (df_cars.Year > 2000)].Color.value_counts()

White     2769
Silver    2087
Black     2064
Grey      1590
Blue      1244
Red        906
Green      397
Brown      323
Custom     212
Yellow     100
Orange      56
Purple      46
Name: Color, dtype: int64